<a href="https://colab.research.google.com/github/gvosper25/3009ICT-Assignmnet/blob/main/3009ICT_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Processing and Visualization
-----
**Assignment Topic: Data Analysis on Car Advertisements**


Student1 Name: Georgia Vosper

SNumber: S5223182


# Data Import

In [2]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import sqlite3
import seaborn as sns
from google.colab import drive
drive.mount ('/content/drive')

Mounted at /content/drive


In [3]:
car_data = pd.read_csv('/content/drive/MyDrive/3009ICT-Assignment/car_dataset.csv')
sellers_data = pd.read_csv('/content/drive/MyDrive/3009ICT-Assignment/seller_dataset.csv')

## a. Statistics Computation

# Data Formatting

## a. Numerical Conversion

List of colums that need to be cleaned & converted





In [18]:
cols_to_clean = [ 'mileage','num_of_doors','seating_capacity','fuel_consumption','price (AUD)']

Loops through each colums & extracts numbers from text. Converts to float

In [19]:
for col in cols_to_clean:
  car_data[col] = car_data[col].astype(str).str.extract(r'([\d.]+)').astype(float)

Run as a test

In [20]:
car_data[['mileage' , 'num_of_doors' , 'seating_capacity' , 'fuel_consumption' , 'price (AUD)']].head ()

,mileage,num_of_doors,seating_capacity,fuel_consumption,price (AUD)
0,0.0,2.0,2.0,100.0,10080.99411
1,0.0,5.0,7.0,10.0,161943.68050
2,0.0,5.0,8.0,100.0,35830.03930
3,0.0,5.0,5.0,100.0,30526.38376
4,0.0,5.0,8.0,100.0,34413.03209


## b. Engine Feature Split

Split into type & size

In [24]:
car_data[['type_of_engine','engine_capacity']]=car_data['engine'].str.extract(r'(\w+)\s*[\t]*\s*([\d.]+)\s*L')

Convert to float

In [25]:
car_data['engine_capacity']=car_data['engine_capacity'].astype(float)

Test

In [26]:
car_data[['engine','type_of_engine','engine_capacity']].head()

,engine,type_of_engine,engine_capacity
0,Petrol\t1.0 L,Petrol,1.0
1,Petrol\t3.4 L,Petrol,3.4
2,Petrol\t2.0 L,Petrol,2.0
3,Petrol\t1.8 L,Petrol,1.8
4,Petrol\t2.0 L,Petrol,2.0


## c. Standardizing Text Data

In [10]:
... # Your code goes here

Ellipsis

## d. Numerical Scaling

In [11]:
... # Your code goes here

Ellipsis

# Data Exploration and Cleaning

## a. Statistics Computation

In [16]:
car_data[['mileage' , 'num_of_doors' , 'seating_capacity' , 'fuel_consumption' , 'price (AUD)']].head ()

,mileage,num_of_doors,seating_capacity,fuel_consumption,price (AUD)
0,0.0,2.0,2.0,100.0,10080.99411
1,0.0,5.0,7.0,10.0,161943.68050
2,0.0,5.0,8.0,100.0,35830.03930
3,0.0,5.0,5.0,100.0,30526.38376
4,0.0,5.0,8.0,100.0,34413.03209


## b. Outliers and Missing Values

In [17]:
... # Your code goes here

Ellipsis

# Data Storage

In [12]:
... # Your code goes here

Ellipsis

# Data Visualization

In [13]:
... # Create as many cells as you need

Ellipsis